In [1]:
import os
import requests
import datetime

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup

from typing import Callable

In [2]:
def pandas_settings() -> None:
        for option in ('display.max_rows', 'display.max_columns', 'display.width', 'display.memory_usage'):
            pd.set_option(option, 250 if 'memory_usage' not in option else False)
        # message('pandas')
        return None
pandas_settings()

In [3]:
class Conversions:
    
    def __init__(self):
        
        self.inits_issues: dict[str,str] = {
            'SAS': 'SA',
            'PHX': 'PHO',
            'GSW': 'GS',
            'NOP': 'NO'
        }
        
        self.inits_teams: dict[str, str] = {
            'NY': 'New York Knicks',
            'LAL': 'Los Angeles Lakers',
            'MIA': 'Miami Heat',
            'UTA': 'Utah Jazz',
            'PHO': 'Phoenix Suns',
            'LAC': 'Los Angeles Clippers',
            # 'LAC': 'LA Clippers',
            'PHI': 'Philadelphia 76ers',
            'DAL': 'Dallas Mavericks',
            'DEN': 'Denver Nuggets',
            'BOS': 'Boston Celtics',
            'ATL': 'Atlanta Hawks',
            'CLE': 'Cleveland Cavaliers',
            'DET': 'Detroit Pistons',
            'TOR': 'Toronto Raptors',
            'CHA': 'Charlotte Hornets',
            'ORL': 'Orlando Magic',
            'MEM': 'Memphis Grizzlies',
            'SA': 'San Antonio Spurs',
            'MIL': 'Milwaukee Bucks',
            'IND': 'Indiana Pacers',
            'CHI': 'Chicago Bulls',
            'OKC': 'Oklahoma City Thunder',
            'GS': 'Golden State Warriors',
            'HOU': 'Houston Rockets',
            'BKN': 'Brooklyn Nets',
            'POR': 'Portland Trail Blazers',
            'NO': 'New Orleans Pelicans',
            'MIN': 'Minnesota Timberwolves',
            'SAC': 'Sacramento Kings',
            'WAS': 'Washington Wizards'
        }

        # Invert
        self.teams_inits: dict[str,str] = { val: key for key, val in self.inits_teams.items() }
        
#         scoresandodds.com: FanDuel name
        self.name_issues: dict[str,str] = {
            'Lu Dort': 'Luguentz Dort',
            'Moe Wagner': 'Moritz Wagner',
            'KJ Martin': 'Kenyon Martin',
            'Devonte Graham': "Devonte' Graham"
        }
    
        
    def team_name(self, team_str: str) -> str:
        return self.teams_inits[team_str]
    
    def team_initials(self, team_init_str: str) -> str:
        return self.inits_teams[team_init_str]
    
    def player_name(self, name: str):
        return self.name_issues.get(name,name)
    
    def initals_issue(self, team_inits: str) -> str:
        return self.inits_issues.get(team_inits,team_inits)

In [4]:
class PropsScraper:
    
    def __init__(self):
        self.convert = Conversions()
        self.directory_url: str = 'https://www.scoresandodds.com/nba/players'
        
        self.current_date_str = datetime.datetime.now().strftime("%m/%d")
        
    
#     Creates a dictionary containing the links to current and historical props
#     for every player in the NBA, organized by team
    def create_webpage_directory(self) -> dict[str, dict[str, str]]:
        
#         Load HTML into bs4
        soup = BeautifulSoup(
            requests.get(self.directory_url).text,
            'html.parser'
        )

#         Load each team data into dictionary, converting the full team name into initials as used in rest of data
        team_modules = {
            self.convert.team_name(team_html.find('h3').get_text()): team_html.find_all('div', class_='module-body')[0].find('ul')
            for team_html in soup.find_all('div', class_='module')
        }
        
        
        clean_name: Callable[[str],str] = lambda name: self.convert.player_name(' '.join(name.split(' ')[:2]).replace('.', ''))
        
#         Parse HTML data for each team to organize links in easily searchable manner
        teams_players_links: dict[str, dict[str, str]] = {
            
            team: {
                clean_name(a_tag.get_text()): self.directory_url.replace(
                    '/nba/players',
                    a_tag['href']
                )
                for a_tag in module.find_all('a')
            }
            
            for team, module in team_modules.items()
            
        }
        
        return teams_players_links
    
    # Implied Probability = 100 / (Odds + 100)
    @staticmethod
    def pos_ml_prob(ml: str) -> float:
        return 100 / sum([int(ml[1:]),100])
    
    # Implied Probability = (-1*(Odds)) / (-1(Odds) + 100) ->
    @staticmethod
    def neg_ml_prob(ml: str) -> float:
        ml: int = int(ml)
        return (-1*ml) / sum([-1*ml,100])
        
    @classmethod
    def implied_probability(cls, ml: str):
        if ml == '+100':
            return 0.5
        
        return cls.pos_ml_prob(ml) if ml[0]=='+' else cls.neg_ml_prob(ml)
    
    @classmethod
    def expected_value(cls, val: float, ml: str) -> float:
        return cls.implied_probability(ml)*val
        
    def scrape_player_props(
        self, 
        name: str, 
        url: str, 
        mode: str
    ) -> tuple[float,float]:
        
#         Load HTML
        soup = BeautifulSoup(
            requests.get(url).text, 
            'html.parser'
        )
        
        module = soup.find('div', class_="module-body scroll")
        
        if not len(module.find_all('span')):
            return (0.0,0.0)
        
#         Make sure current
        zerofill = lambda dp: f'0{dp}' if len(dp) == 1 else dp
        date_str = '/'.join([
            zerofill(dp) for dp in module.find_all('span')[2].get_text().split(' ')[1].split('/')
        ])
        
        if date_str != self.current_date_str:
            return (0.0,0.0)
        
        props_rows = module.find('table', class_='sticky').find('tbody').find_all('tr')
        
        # Steals, blocks are options but noisy, better to use season data for opponents
        
        site_targets: dict[str,tuple[str,...]] = {
            'fanduel': ('Points', 'Rebounds', 'Assists', 'Steals', 'Blocks'),
            'draftkings': ('Points', 'Rebounds', 'Assists', '3 Pointers')
        }
        
        targets = site_targets[mode]
        
#         Form: Category Line Over Under
        target_rows = [row for row in props_rows if row.find('td').get_text() in targets]
    
    
#     TODO: Figure out more efficient way for this, dict(zip()) probably best
        props = {
            'stat': list(),
            'value': list(),
            # 'over': list(),
            'e_value': list(),
            'fpts': list(),
            'e_fpts': list()
            # 'under': list()
        }
        
        
        site_multipliers: dict[str,dict[str,float]] = {
            'fanduel': {'assists': 1.5, 'rebounds': 1.2, 'blocks': 3.0, 'steals': 3.0, '3 pointers': 0.0},
            'draftkings': {'assists': 1.5, 'rebounds': 1.25, '3 pointers': 0.5} #, 'blocks': 2.0, 'steals': 2.0, }
        }
        
        multipliers: dict[str,float] = site_multipliers[mode]
        for rowtags in target_rows:
            vals = [val.get_text().lower() for val in rowtags.find_all('td')] # (Category, Line, Over, Under)
            
            stat: str = vals[0]
            props['stat'].append(stat)
            
            statval = sum([float(vals[1]), 0.5])
            props['value'].append(statval)
            
            overml: str = vals[2]
            
            props['e_value'].append(self.expected_value(statval, overml))
            
            multi: float = multipliers.get(stat, 1.0)
            fpts: float = multi*statval
            
            props['fpts'].append(fpts)
            props['e_fpts'].append(self.expected_value(fpts, overml))
            # props['under'].append(vals[3])
        
        
        df: pd.DataFrame = pd.DataFrame(props).round(2)
        
        
        return (df['fpts'].sum(), df['e_fpts'].sum())

In [5]:
# Incomplete rn
Props = PropsScraper()
directory: dict[str,dict[str,str]] = Props.create_webpage_directory()

def test_(name: str, team: str, **kwargs):
    
#     url: str = 
#     mode: str = 
    
    return Props.scrape_player_props(
        name,
        directory[team][name],
        kwargs.get('mode', 'fanduel')
    )
    
    # return (fp, efp) #if kwargs.get('salary') is None else tuple([ 1000 * (fp_/kwargs['salary']) for fp_ in (fp,efp) ])

In [6]:
# issues = {
#         'Luguentz Dort': 'Lu Dort', 
#         'Moritz Wagner':'Moe Wagner',
#         'Kenyon Martin': 'KJ Martin',
#         "Devonte' Graham": "Devonte Graham"
# }

# {val: key for key,val in issues.items()}

In [7]:
def load_fanduel_slate():
    # path: str = f'../data/current-{mode}.csv'
    path: str = '../data/current-fanduel.csv'
    
    columns: dict[str, str] = {
        'Nickname': 'name',
        'Position': 'pos',
        'Team': 'team',
        'Salary': 'salary'
    }
    
    ignores: tuple[str,...] = ('Isaiah Roby',)
    
    df: pd.DataFrame = (pd
                        .read_csv(path, usecols=columns)
                        .rename(columns,axis=1)
                        .pipe(lambda df_: df_.loc[df_['salary']>3500])
                        .pipe(lambda df_: df_.loc[df_['name'].isin(ignores) == False])
                        .assign(name=lambda df_: df_.name.str.replace('.','',regex=False))
                       )
    
    df['input'] = tuple(zip(df['name'], df['team']))
    df['output'] = df['input'].apply(lambda x: test_(*x))
    
    df['fpts'] = df['output'].map(lambda x: x[0])
    df['e_fpts'] = df['output'].map(lambda x: x[1])
    
    
    for col in ('fpts', 'e_fpts'):
        df[f'{col}/$'] = 1000 * (df[col] / df['salary'])
    
    df['5x'] = 5 * (df['salary'] / 1000)
    df['value'] = df['fpts'] - df['5x']
    
    df = (df
          .loc[df['fpts']>0.0]
          .drop(['input', 'output', '5x'], axis=1)
          .sort_values('value', ascending=False)
          .set_index('name')
          .round(2)
         )
    
    df.to_csv('../data/fanduel-props.csv')
    
    return df

In [8]:
def load_draftkings_slate():

    path: str = '../data/current-draftkings.csv'
    
    columns: dict[str, str] = {
        'Name': 'name',
        'Roster Position': 'pos',
        'TeamAbbrev': 'team',
        'Salary': 'salary'
    }
    
    inits_issues = {
        'SAS': 'SA',
        'PHX': 'PHO',
        'GSW': 'GS',
        'NOP': 'NO'
    }
    
    ignores: tuple[str,...] = ('Isaiah Roby',)
    
    df: pd.DataFrame = (pd
                        .read_csv(path, usecols=columns)
                        .rename(columns,axis=1)
                        .pipe(lambda df_: df_.loc[df_['salary']>3000])
                        .pipe(lambda df_: df_.loc[df_['name'].isin(ignores) == False])
                        .assign(
                            name=lambda df_: df_.name.str.replace('.','', regex=False),
                            pos=lambda df_: df_.pos.str.replace('/[GF]/UTIL','', regex=True)
                        )
                       )
    
    name_issues: dict[str,str] = {
        'KJ Martin': 'Kenyon Martin',
    }
    
    df['name'] = df['name'].map(lambda x: ' '.join(x.split(' ')[:2]))
    df['name'] = df['name'].map(lambda x: name_issues.get(x,x))
    
    df['team'] = df['team'].map(lambda x: inits_issues.get(x,x))
    
    df['input'] = tuple(zip(df['name'], df['team']))
    df['output'] = df['input'].apply(lambda x: test_(*x))
    
    df['fpts'] = df['output'].map(lambda x: x[0])
    df['e_fpts'] = df['output'].map(lambda x: x[1])
    
    
    for col in ('fpts', 'e_fpts'):
        df[f'{col}/$'] = 1000 * (df[col] / df['salary'])
    
    df['5x'] = 5 * (df['salary'] / 1000)
    df['value'] = df['fpts'] - df['5x']
    
    df = (df
          .loc[df['fpts']>0.0]
          .drop(['input', 'output', '5x'], axis=1)
          .sort_values('value', ascending=False)
          .set_index('name')
          .round(2)
         )
    
    df.to_csv('../data/draftkings-props.csv')
    
    return df

In [9]:
# Temporary, just figuring out dynamics for now

def load_slate(**kwargs):
    
    mode: str = kwargs.get('mode', 'fanduel')
    
    return load_fanduel_slate() if mode == 'fanduel' else load_draftkings_slate()
    

In [ ]:
load_slate()

In [ ]:
(pd
 .read_csv('../data/fanduel-props.csv')
 .set_index('name')
 .sort_values('e_fpts/$', ascending=False)
 # .sort_values(['pos', 'e_fpts/$'], ascending=[True, False])
 # .sort_values([ 'salary', 'e_fpts/$'], ascending=[True, False])
)

In [ ]:
# load_slate(mode='draftkings')

In [ ]:
# (pd
#  .read_csv('../data/draftkings-props.csv')
#  .set_index('name')
#  .sort_values('e_fpts/$', ascending=False)
#  # .sort_values(['pos', 'e_fpts/$'], ascending=[True, False])
#  # .sort_values([ 'salary', 'e_fpts/$'], ascending=[True, False])
# )

In [20]:
def compare_players(*args, **kwargs):
    return (pd
            .read_csv(f'../data/{kwargs.get("mode", "fanduel")}-props.csv')
            .set_index('name')
            .sort_values('value', ascending=False)
            .pipe(lambda df_: df_.loc[df_.index.isin(args)])
           )

In [33]:
def payup_options(**kwargs):
    

    #Add df.query
    df: pd.DataFrame = (pd
                        .read_csv(f'../data/{kwargs.get("mode", "fanduel")}-props.csv')
                        .set_index('name')
                       )
    
    #Add better selector for site
    names: tuple[str,...] = tuple(df
                                  .loc[df['salary']>=kwargs.get('salary', 8000)]
                                  .index
                                 )
    
    return compare_players(*names,**kwargs).sort_values('e_fpts/$', ascending=False)

In [39]:
payup_options()

,pos,salary,team,fpts,e_fpts,fpts/$,e_fpts/$,value
name,,,,,,,,
Giannis Antetokounmpo,PF/SF,11600,MIL,55.4,30.37,4.78,2.62,-2.6
De'Aaron Fox,PG,9300,SAC,44.5,24.16,4.78,2.60,-2.0
James Harden,SG/PG,9900,PHI,47.9,24.14,4.84,2.44,-1.6
Domantas Sabonis,C/PF,9500,SAC,45.1,23.18,4.75,2.44,-2.4
Trae Young,PG,9700,ATL,44.6,23.58,4.60,2.43,-3.9
Joel Embiid,C,11700,PHI,51.2,27.90,4.38,2.38,-7.3
Anthony Edwards,SG/SF,8900,MIN,40.2,20.97,4.52,2.36,-4.3
Bam Adebayo,C,8300,MIA,36.8,19.57,4.43,2.36,-4.7
Darius Garland,PG,9000,CLE,38.6,20.71,4.29,2.30,-6.4
